In [ ]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from ipywidgets import interact, FloatSlider, Layout, Tab, VBox, HBox
from IPython.display import display, Markdown, HTML
import plotly.express as px
from scipy.optimize import fsolve

class SolowGrowthModel:
    def __init__(self, params):
        """Initialize Solow Growth Model with parameters"""
        self.params = params
        self.T = params['T']
        self.setup_arrays()
        self.calculate_steady_state()
    
    def setup_arrays(self):
        """Initialize arrays for time series"""
        self.k = np.zeros(self.T)  # Capital per effective worker
        self.y = np.zeros(self.T)  # Output per effective worker
        self.c = np.zeros(self.T)  # Consumption per effective worker
        self.i = np.zeros(self.T)  # Investment per effective worker
        self.growth_rate = np.zeros(self.T-1)  # Growth rate of output
        self.k[0] = self.params['k0']
    
    def calculate_steady_state(self):
        """Calculate steady state values"""
        p = self.params
        self.k_star = (p['s'] / (p['delta'] + p['g_A'] + p['g_L'])) ** (1 / (1 - p['alpha']))
        self.y_star = self.k_star ** p['alpha']
        self.i_star = (p['delta'] + p['g_A'] + p['g_L']) * self.k_star
        self.c_star = self.y_star - self.i_star
        
        # Calculate golden rule
        self.k_golden = ((p['alpha']) / (p['delta'] + p['g_A'] + p['g_L'])) ** (1 / (1 - p['alpha']))
        self.y_golden = self.k_golden ** p['alpha']
        self.c_golden = self.y_golden - (p['delta'] + p['g_A'] + p['g_L']) * self.k_golden
    
    def simulate(self):
        """Run the Solow model simulation"""
        p = self.params
        
        for t in range(1, self.T):
            # Production
            self.y[t-1] = self.k[t-1] ** p['alpha']
            
            # Saving and Investment
            self.i[t-1] = p['s'] * self.y[t-1]
            self.c[t-1] = self.y[t-1] - self.i[t-1]
            
            # Capital Accumulation
            depreciation = (p['delta'] + p['g_A'] + p['g_L']) * self.k[t-1]
            self.k[t] = self.k[t-1] + self.i[t-1] - depreciation
            
            # Growth Rate
            if t > 1:
                self.growth_rate[t-2] = (self.y[t-1] - self.y[t-2]) / self.y[t-2] * 100
        
        # Final period
        self.y[-1] = self.k[-1] ** p['alpha']
        self.i[-1] = p['s'] * self.y[-1]
        self.c[-1] = self.y[-1] - self.i[-1]
    
    def plot_simulation(self):
        """Create comprehensive visualization of simulation results"""
        time = np.arange(self.T)
        
        # Create figure with secondary y-axis
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Capital and Output per Effective Worker',
                'Growth Rate Dynamics',
                'Saving and Consumption',
                'Phase Diagram'
            )
        )
        
        # Plot 1: Capital and Output
        fig.add_trace(
            go.Scatter(x=time, y=self.k, name='Capital', line=dict(color='blue')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=time, y=self.y, name='Output', line=dict(color='red')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=[0, self.T], y=[self.k_star, self.k_star],
                      name='Steady State k*', line=dict(dash='dash', color='gray')),
            row=1, col=1
        )
        
        # Plot 2: Growth Rate
        fig.add_trace(
            go.Scatter(x=time[1:-1], y=self.growth_rate, name='Growth Rate',
                      line=dict(color='green')),
            row=1, col=2
        )
        
        # Plot 3: Saving and Consumption
        fig.add_trace(
            go.Scatter(x=time, y=self.c, name='Consumption',
                      line=dict(color='purple')),
            row=2, col=1
        )
        fig.add_trace(
            go.Scatter(x=time, y=self.i, name='Investment',
                      line=dict(color='orange')),
            row=2, col=1
        )
        
        # Plot 4: Phase Diagram
        k_range = np.linspace(0, max(self.k) * 1.2, 100)
        sf_k = self.params['s'] * k_range ** self.params['alpha']
        dep_k = (self.params['delta'] + self.params['g_A'] + self.params['g_L']) * k_range
        
        fig.add_trace(
            go.Scatter(x=k_range, y=sf_k, name='sf(k)',
                      line=dict(color='blue')),
            row=2, col=2
        )
        fig.add_trace(
            go.Scatter(x=k_range, y=dep_k, name='(δ+g)k',
                      line=dict(color='red')),
            row=2, col=2
        )
        fig.add_trace(
            go.Scatter(x=[self.k_star], y=[self.params['s'] * self.k_star ** self.params['alpha']],
                      mode='markers', name='Steady State',
                      marker=dict(size=10, color='black')),
            row=2, col=2
        )
        
        # Update layout
        fig.update_layout(height=800, width=1200, showlegend=True,
                         title_text="Solow Growth Model Analysis")
        
        # Update axes labels
        fig.update_xaxes(title_text="Time", row=1, col=1)
        fig.update_xaxes(title_text="Time", row=1, col=2)
        fig.update_xaxes(title_text="Time", row=2, col=1)
        fig.update_xaxes(title_text="Capital per Effective Worker", row=2, col=2)
        
        fig.update_yaxes(title_text="Level", row=1, col=1)
        fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=2)
        fig.update_yaxes(title_text="Level", row=2, col=1)
        fig.update_yaxes(title_text="Investment/Depreciation", row=2, col=2)
        
        fig.show()
        
        # Display analysis
        self.display_analysis()
    
    def display_analysis(self):
        """Display comprehensive analysis of simulation results"""
        # Calculate convergence metrics
        half_life = self.calculate_half_life()
        convergence_speed = self.calculate_convergence_speed()
        
        analysis = f"""
        ### 📊 Solow Model Analysis
        
        #### 1. Steady State Values:
        - Capital per effective worker (k*): {self.k_star:.3f}
        - Output per effective worker (y*): {self.y_star:.3f}
        - Consumption per effective worker (c*): {self.c_star:.3f}
        
        #### 2. Golden Rule Analysis:
        - Golden Rule capital (k_g): {self.k_golden:.3f}
        - Golden Rule consumption (c_g): {self.c_golden:.3f}
        - Current vs. Golden Rule: {"Above" if self.k_star > self.k_golden else "Below"}
        
        #### 3. Convergence Analysis:
        - Half-life to steady state: {half_life:.1f} periods
        - Speed of convergence: {convergence_speed:.3f}
        - Time to 95% of steady state: {self.time_to_threshold(0.95):.1f} periods
        
        #### 4. Policy Implications:
        {self.generate_policy_recommendations()}
        """
        
        display(Markdown(analysis))
    
    def calculate_half_life(self):
        """Calculate time to reach half of steady state deviation"""
        initial_gap = abs(self.k[0] - self.k_star)
        half_gap = initial_gap / 2
        
        for t, k in enumerate(self.k):
            if abs(k - self.k_star) <= half_gap:
                return t
        
        return self.T
    
    def calculate_convergence_speed(self):
        """Calculate speed of convergence to steady state"""
        log_k_ratio = np.log(self.k[1:] / self.k[:-1])
        return -np.mean(log_k_ratio) * 100
    
    def time_to_threshold(self, threshold):
        """Calculate time to reach threshold percentage of steady state"""
        target_gap = (1 - threshold) * abs(self.k[0] - self.k_star)
        
        for t, k in enumerate(self.k):
            if abs(k - self.k_star) <= target_gap:
                return t
        
        return self.T
    
    def generate_policy_recommendations(self):
        """Generate policy recommendations based on simulation results"""
        recommendations = []
        
        # Check if economy is at golden rule
        if abs(self.k_star - self.k_golden) / self.k_golden > 0.1:
            if self.k_star > self.k_golden:
                recommendations.append("Economy is dynamically inefficient. Consider policies to reduce saving rate.")
            else:
                recommendations.append("Economy is below golden rule. Consider policies to increase saving rate.")
        
        # Check convergence speed
        if self.calculate_convergence_speed() < 0.02:
            recommendations.append("Slow convergence suggests need for structural reforms to increase efficiency.")
        
        # Check consumption level
        if min(self.c) < 0.8 * self.c_star:
            recommendations.append("Low consumption during transition suggests need for social safety nets.")
        
        return "- " + "\n- ".join(recommendations)

# Create interactive simulation
def run_interactive_simulation(
    s=0.2, delta=0.05, g_A=0.02, g_L=0.01, alpha=0.33, k0=0.5, T=100
):
    """Run interactive Solow model simulation"""
    params = {
        's': s,
        'delta': delta,
        'g_A': g_A,
        'g_L': g_L,
        'alpha': alpha,
        'k0': k0,
        'T': T
    }
    
    model = SolowGrowthModel(params)
    model.simulate()
    model.plot_simulation()

# Create interactive widgets with improved layout
interact(
    run_interactive_simulation,
    s=FloatSlider(value=0.2, min=0.05, max=0.6, step=0.01,
                  description='Savings Rate (s)',
                  style={'description_width': 'initial'},
                  layout=Layout(width='500px')),
    delta=FloatSlider(value=0.05, min=0.01, max=0.2, step=0.005,
                      description='Depreciation (δ)',
                      style={'description_width': 'initial'},
                      layout=Layout(width='500px')),
    g_A=FloatSlider(value=0.02, min=0.0, max=0.05, step=0.005,
                    description='Tech Growth (gₐ)',
                    style={'description_width': 'initial'},
                    layout=Layout(width='500px')),
    g_L=FloatSlider(value=0.01, min=0.0, max=0.05, step=0.005,
                    description='Pop Growth (gₗ)',
                    style={'description_width': 'initial'},
                    layout=Layout(width='500px')),
    alpha=FloatSlider(value=0.33, min=0.2, max=0.5, step=0.01,
                      description='Capital Share (α)',
                      style={'description_width': 'initial'},
                      layout=Layout(width='500px')),
    k0=FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1,
                   description='Initial Capital (k₀)',
                   style={'description_width': 'initial'},
                   layout=Layout(width='500px'))
)

interactive(children=(FloatSlider(value=0.2, description='Savings Rate (s)', max=0.6, min=0.05, step=0.01), Fl…

<function __main__.solow_simulation(s=0.2, delta=0.05, g_A=0.02, g_L=0.01, alpha=0.33, T=100, k0=0.5)>

In [ ]:
class CountryGrowthAnalysis:
    def __init__(self):
        """Initialize with sample cross-country data"""
        self.data = self.load_country_data()
        
    def load_country_data(self):
        """Load and process cross-country economic data"""
        # Create sample data (replace with actual World Bank/Penn World Table data)
        countries = ['USA', 'China', 'Japan', 'Germany', 'India', 'UK', 'France', 'Brazil']
        base_year = 2000
        years = range(base_year, 2024)
        
        data = []
        np.random.seed(42)
        
        for country in countries:
            # Generate country-specific parameters
            if country in ['USA', 'UK', 'Germany', 'France', 'Japan']:  # Advanced economies
                g_base = 0.02
                s_base = 0.22
                volatility = 0.01
            else:  # Emerging economies
                g_base = 0.04
                s_base = 0.30
                volatility = 0.02
            
            # Generate time series
            gdp_base = 100 * (1 + np.random.normal(0, 0.2))  # Random initial GDP
            s = s_base + np.random.normal(0, 0.05)  # Country-specific saving rate
            
            for year in years:
                growth = g_base + np.random.normal(0, volatility)
                gdp = gdp_base * np.exp(growth * (year - base_year))
                
                data.append({
                    'Country': country,
                    'Year': year,
                    'GDP_per_capita': gdp,
                    'Saving_Rate': s + np.random.normal(0, 0.01),
                    'Investment_Rate': s + np.random.normal(0, 0.01),
                    'TFP_Growth': g_base/2 + np.random.normal(0, volatility/2),
                    'Capital_per_Worker': gdp * (s/(0.05 + g_base))**(1/0.33)
                })
                
        return pd.DataFrame(data)
    
    def plot_cross_country_analysis(self):
        """Create comprehensive cross-country comparison plots"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'GDP per Capita Growth',
                'Saving Rates Across Countries',
                'Capital-Output Ratios',
                'Growth Accounting'
            )
        )
        
        # Plot 1: GDP per Capita Growth
        for country in self.data['Country'].unique():
            country_data = self.data[self.data['Country'] == country]
            fig.add_trace(
                go.Scatter(
                    x=country_data['Year'],
                    y=country_data['GDP_per_capita'],
                    name=country,
                    mode='lines'
                ),
                row=1, col=1
            )
        
        # Plot 2: Saving Rates
        fig.add_trace(
            go.Box(
                x=self.data['Country'],
                y=self.data['Saving_Rate'],
                name='Saving Rates'
            ),
            row=1, col=2
        )
        
        # Plot 3: Capital-Output Ratio Evolution
        self.data['Capital_Output_Ratio'] = self.data['Capital_per_Worker'] / self.data['GDP_per_capita']
        for country in self.data['Country'].unique():
            country_data = self.data[self.data['Country'] == country]
            fig.add_trace(
                go.Scatter(
                    x=country_data['Year'],
                    y=country_data['Capital_Output_Ratio'],
                    name=country
                ),
                row=2, col=1
            )
        
        # Plot 4: Growth Accounting
        growth_components = self.calculate_growth_components()
        fig.add_trace(
            go.Bar(
                x=growth_components.index,
                y=growth_components['TFP_Contribution'],
                name='TFP Contribution'
            ),
            row=2, col=2
        )
        
        fig.add_trace(
            go.Bar(
                x=growth_components.index,
                y=growth_components['Capital_Contribution'],
                name='Capital Contribution'
            ),
            row=2, col=2
        )
        
        # Update layout
        fig.update_layout(
            height=800,
            width=1200,
            title_text="Cross-Country Growth Analysis",
            showlegend=True,
            barmode='stack'
        )
        
        # Update axes labels
        fig.update_xaxes(title_text="Year", row=1, col=1)
        fig.update_xaxes(title_text="Country", row=1, col=2)
        fig.update_xaxes(title_text="Year", row=2, col=1)
        fig.update_xaxes(title_text="Country", row=2, col=2)
        
        fig.update_yaxes(title_text="GDP per Capita", row=1, col=1)
        fig.update_yaxes(title_text="Saving Rate", row=1, col=2)
        fig.update_yaxes(title_text="Capital-Output Ratio", row=2, col=1)
        fig.update_yaxes(title_text="Growth Contribution (%)", row=2, col=2)
        
        fig.show()
        
        # Display analysis
        self.display_cross_country_analysis()
    
    def calculate_growth_components(self):
        """Calculate growth accounting components"""
        results = []
        
        for country in self.data['Country'].unique():
            country_data = self.data[self.data['Country'] == country]
            
            # Calculate average growth rates
            tfp_growth = country_data['TFP_Growth'].mean()
            capital_growth = (country_data['Capital_per_Worker'].pct_change().mean())
            
            # Assume capital share of 1/3
            capital_contribution = 0.33 * capital_growth
            tfp_contribution = tfp_growth
            
            results.append({
                'Country': country,
                'TFP_Contribution': tfp_contribution * 100,
                'Capital_Contribution': capital_contribution * 100
            })
        
        return pd.DataFrame(results).set_index('Country')
    
    def display_cross_country_analysis(self):
        """Display comprehensive cross-country analysis"""
        # Calculate summary statistics
        summary = pd.DataFrame({
            'Avg_Growth': self.data.groupby('Country')['GDP_per_capita'].pct_change().mean(),
            'Avg_Saving': self.data.groupby('Country')['Saving_Rate'].mean(),
            'Avg_TFP_Growth': self.data.groupby('Country')['TFP_Growth'].mean(),
            'K_Y_Ratio': self.data.groupby('Country')['Capital_Output_Ratio'].mean()
        })
        
        analysis = f"""
        ### 📊 Cross-Country Growth Analysis
        
        #### 1. Growth Performance (2000-2023):
        - Highest Average Growth: {summary['Avg_Growth'].idxmax()} ({summary['Avg_Growth'].max()*100:.1f}%)
        - Lowest Average Growth: {summary['Avg_Growth'].idxmin()} ({summary['Avg_Growth'].min()*100:.1f}%)
        
        #### 2. Saving and Investment:
        - Highest Saving Rate: {summary['Avg_Saving'].idxmax()} ({summary['Avg_Saving'].max()*100:.1f}%)
        - Lowest Saving Rate: {summary['Avg_Saving'].idxmin()} ({summary['Avg_Saving'].min()*100:.1f}%)
        
        #### 3. Capital Intensity:
        - Highest K/Y Ratio: {summary['K_Y_Ratio'].idxmax()} ({summary['K_Y_Ratio'].max():.2f})
        - Lowest K/Y Ratio: {summary['K_Y_Ratio'].idxmin()} ({summary['K_Y_Ratio'].min():.2f})
        
        #### 4. Productivity Growth:
        - Highest TFP Growth: {summary['Avg_TFP_Growth'].idxmax()} ({summary['Avg_TFP_Growth'].max()*100:.1f}%)
        - Lowest TFP Growth: {summary['Avg_TFP_Growth'].idxmin()} ({summary['Avg_TFP_Growth'].min()*100:.1f}%)
        
        #### 5. Key Insights:
        - {self.generate_insights(summary)}
        """
        
        display(Markdown(analysis))
    
    def generate_insights(self, summary):
        """Generate key insights from the cross-country analysis"""
        insights = []
        
        # Growth convergence
        emerging = ['China', 'India', 'Brazil']
        advanced = ['USA', 'UK', 'Germany', 'France', 'Japan']
        
        emerging_growth = summary.loc[emerging, 'Avg_Growth'].mean()
        advanced_growth = summary.loc[advanced, 'Avg_Growth'].mean()
        
        if emerging_growth > advanced_growth:
            insights.append("Evidence of growth convergence with emerging economies growing faster")
        else:
            insights.append("Limited evidence of growth convergence")
        
        # Saving rates and growth
        corr = summary[['Avg_Growth', 'Avg_Saving']].corr().iloc[0,1]
        if abs(corr) > 0.3:
            insights.append(f"Strong {'positive' if corr > 0 else 'negative'} correlation between saving rates and growth")
        
        # TFP contribution
        tfp_share = summary['Avg_TFP_Growth'] / (summary['Avg_Growth'])
        if tfp_share.mean() > 0.5:
            insights.append("TFP growth is the dominant source of growth for most countries")
        else:
            insights.append("Capital accumulation remains important for growth")
        
        return "\n- ".join(insights)

# Run cross-country analysis
analysis = CountryGrowthAnalysis()
analysis.plot_cross_country_analysis()

# 🎯 Advanced Exercises and Case Studies

## 📊 Case Study 1: East Asian Growth Miracle

### Background
The rapid growth of East Asian economies (Japan, South Korea, Singapore, Hong Kong) from the 1960s to 1990s provides a perfect laboratory for studying the Solow model's predictions.

1. **Key Features**
   - High saving rates (30-40% of GDP)
   - Rapid capital accumulation
   - Strong TFP growth
   - Export-oriented industrialization

2. **Analysis Tasks**
   a) Use the Solow simulator to replicate the growth path with:
      - High saving rate (s = 0.35)
      - Moderate depreciation (δ = 0.05)
      - Strong technological progress (gₐ = 0.03)
   
   b) Compare model predictions with actual data:
      - How much of the growth was due to capital deepening?
      - Did diminishing returns eventually slow growth?
      - What role did TFP play?

3. **Policy Questions**
   - Was the high saving rate optimal?
   - Could other countries replicate this success?
   - What explains the post-1990 slowdown?

## 📈 Case Study 2: Growth Accounting in Practice

### Project Guidelines

1. **Data Collection**
   ```python
   def collect_growth_data(country):
       """Template for collecting growth accounting data"""
       data = {
           'GDP_growth': [],
           'Capital_growth': [],
           'Labor_growth': [],
           'TFP_growth': []
       }
       return pd.DataFrame(data)
   ```

2. **Growth Decomposition**
   - Calculate factor contributions
   - Estimate TFP as residual
   - Compare across countries/periods

3. **Deliverables**
   - Growth accounting tables
   - Factor contribution charts
   - Policy recommendations

## 🏛️ Exercise Set: Advanced Solow Analysis

### Exercise 1: Golden Rule Analysis

1. For an economy with:
   - α = 0.3
   - δ = 0.05
   - g_A = 0.02
   - g_L = 0.01

2. Tasks:
   a) Calculate the golden rule saving rate
   b) Compare steady states under different saving rates
   c) Compute welfare implications of transition

### Exercise 2: Convergence Analysis

1. Using the simulation tools:
   ```python
   def calculate_convergence_speed(k, k_star):
       """Template for convergence analysis"""
       log_ratio = np.log(k / k_star)
       return -np.mean(np.diff(log_ratio))
   ```

2. Compare convergence speeds:
   - Different initial conditions
   - Different parameter values
   - Rich vs. poor countries

### Exercise 3: Policy Evaluation

1. Consider these policies:
   - Investment subsidies
   - Education spending
   - R&D incentives

2. For each policy:
   - Model as parameter changes
   - Simulate transitions
   - Calculate welfare effects

## 🌍 Real-World Applications

### Project 1: Development Accounting

1. Research Question:
   "How much of income differences across countries is explained by capital differences?"

2. Methodology:
   ```python
   def development_accounting(countries_data):
       """Template for development accounting"""
       results = {
           'Capital_share': [],
           'TFP_share': [],
           'Human_capital_share': []
       }
       return results
   ```

3. Analysis:
   - Decompose income differences
   - Test model predictions
   - Policy implications

### Project 2: Growth Diagnostics

1. Framework:
   - Identify binding constraints
   - Evaluate policy options
   - Recommend interventions

2. Case Studies:
   - Successful reformers
   - Growth disasters
   - Middle-income traps

## 📚 Advanced Topics

1. **Extensions to Basic Solow Model**
   - Human capital
   - Multiple sectors
   - Technology diffusion

2. **Empirical Challenges**
   - Measurement issues
   - Identification problems
   - Data quality

3. **Policy Design**
   - Growth-enhancing reforms
   - Structural transformation
   - Institutional development

## 🎓 Further Research

1. **Reading List**
   - Acemoglu (2009): "Introduction to Modern Economic Growth"
   - Jones & Vollrath (2013): "Introduction to Economic Growth"
   - Barro & Sala-i-Martin (2004): "Economic Growth"

2. **Data Sources**
   - Penn World Tables
   - World Bank WDI
   - FRED Economic Data

3. **Research Topics**
   - Growth miracles and disasters
   - Technology adoption
   - Institutional determinants

## 🚀 Next Steps

1. Choose a country/region for in-depth study:
   - Collect detailed data
   - Apply Solow framework
   - Develop policy recommendations

2. Extend the model:
   - Add human capital
   - Include government
   - Model open economy

3. Connect to other topics:
   - Business cycles
   - Income distribution
   - Environmental sustainability

In [ ]:
class GrowthPolicyAnalyzer:
    def __init__(self):
        """Initialize policy analysis tools"""
        self.base_params = {
            's': 0.2,
            'delta': 0.05,
            'g_A': 0.02,
            'g_L': 0.01,
            'alpha': 0.33,
            'A': 1.0,
            'L': 1.0,
            'T': 100
        }
        self.scenarios = self.define_scenarios()
    
    def define_scenarios(self):
        """Define different policy scenarios"""
        return {
            'base': self.base_params.copy(),
            'high_saving': {**self.base_params, 's': 0.3},
            'low_saving': {**self.base_params, 's': 0.1},
            'high_tech': {**self.base_params, 'g_A': 0.04},
            'low_tech': {**self.base_params, 'g_A': 0.01},
            'demographic_decline': {**self.base_params, 'g_L': -0.01},
            'capital_intensive': {**self.base_params, 'alpha': 0.4},
            'labor_intensive': {**self.base_params, 'alpha': 0.25}
        }
    
    def simulate_scenario(self, scenario_params):
        """Simulate a specific policy scenario"""
        k = np.zeros(scenario_params['T'])
        y = np.zeros(scenario_params['T'])
        c = np.zeros(scenario_params['T'])
        tfp = np.zeros(scenario_params['T'])
        
        # Initial conditions
        k[0] = (scenario_params['s'] / (scenario_params['delta'] + 
                scenario_params['g_A'] + scenario_params['g_L'])) ** (
                1 / (1 - scenario_params['alpha'])) * 0.5
        
        # Simulation
        for t in range(1, scenario_params['T']):
            # Production and TFP
            y[t-1] = k[t-1] ** scenario_params['alpha']
            tfp[t-1] = y[t-1] / (k[t-1] ** scenario_params['alpha'])
            
            # Consumption
            c[t-1] = (1 - scenario_params['s']) * y[t-1]
            
            # Capital accumulation
            k[t] = (scenario_params['s'] * y[t-1] + 
                    (1 - scenario_params['delta']) * k[t-1]) / (
                    1 + scenario_params['g_A'] + scenario_params['g_L'])
        
        # Final period
        y[-1] = k[-1] ** scenario_params['alpha']
        tfp[-1] = y[-1] / (k[-1] ** scenario_params['alpha'])
        c[-1] = (1 - scenario_params['s']) * y[-1]
        
        return {'k': k, 'y': y, 'c': c, 'tfp': tfp}
    
    def compare_scenarios(self, scenario_list=None):
        """Compare different policy scenarios"""
        if scenario_list is None:
            scenario_list = list(self.scenarios.keys())
        
        results = {}
        for scenario in scenario_list:
            results[scenario] = self.simulate_scenario(self.scenarios[scenario])
        
        self.plot_scenario_comparison(results, scenario_list)
        self.analyze_welfare_effects(results, scenario_list)
    
    def plot_scenario_comparison(self, results, scenario_list):
        """Create comparative visualization of scenarios"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Output per Effective Worker',
                'Consumption per Effective Worker',
                'Capital per Effective Worker',
                'Total Factor Productivity'
            )
        )
        
        time = np.arange(self.base_params['T'])
        
        # Plot output
        for scenario in scenario_list:
            fig.add_trace(
                go.Scatter(
                    x=time,
                    y=results[scenario]['y'],
                    name=f'{scenario} - Output',
                    line=dict(dash='solid')
                ),
                row=1, col=1
            )
        
        # Plot consumption
        for scenario in scenario_list:
            fig.add_trace(
                go.Scatter(
                    x=time,
                    y=results[scenario]['c'],
                    name=f'{scenario} - Consumption',
                    line=dict(dash='dot')
                ),
                row=1, col=2
            )
        
        # Plot capital
        for scenario in scenario_list:
            fig.add_trace(
                go.Scatter(
                    x=time,
                    y=results[scenario]['k'],
                    name=f'{scenario} - Capital',
                    line=dict(dash='dashdot')
                ),
                row=2, col=1
            )
        
        # Plot TFP
        for scenario in scenario_list:
            fig.add_trace(
                go.Scatter(
                    x=time,
                    y=results[scenario]['tfp'],
                    name=f'{scenario} - TFP',
                    line=dict(dash='solid')
                ),
                row=2, col=2
            )
        
        # Update layout
        fig.update_layout(
            height=800,
            width=1200,
            title_text="Policy Scenario Comparison",
            showlegend=True
        )
        
        fig.show()
    
    def analyze_welfare_effects(self, results, scenario_list):
        """Analyze welfare implications of different scenarios"""
        welfare_analysis = {}
        
        for scenario in scenario_list:
            # Calculate present value of utility
            beta = 0.96  # discount factor
            utility = np.log(results[scenario]['c'])
            pv_utility = np.sum(beta ** np.arange(len(utility)) * utility)
            
            # Calculate steady state values
            ss_y = results[scenario]['y'][-1]
            ss_c = results[scenario]['c'][-1]
            ss_k = results[scenario]['k'][-1]
            
            # Calculate transition speed
            k_star = ss_k
            k_0 = results[scenario]['k'][0]
            half_life = self.calculate_half_life(results[scenario]['k'], k_star)
            
            welfare_analysis[scenario] = {
                'PV_Utility': pv_utility,
                'SS_Output': ss_y,
                'SS_Consumption': ss_c,
                'Half_Life': half_life
            }
        
        self.display_welfare_analysis(welfare_analysis)
    
    def calculate_half_life(self, k, k_star):
        """Calculate half-life of convergence to steady state"""
        initial_gap = abs(k[0] - k_star)
        half_gap = initial_gap / 2
        
        for t, k_t in enumerate(k):
            if abs(k_t - k_star) <= half_gap:
                return t
        
        return len(k)
    
    def display_welfare_analysis(self, welfare_analysis):
        """Display comprehensive welfare analysis"""
        analysis = """
        ### 📊 Policy Scenario Analysis
        
        #### 1. Welfare Comparison (Present Value of Utility):
        """
        
        # Sort scenarios by welfare
        sorted_scenarios = sorted(
            welfare_analysis.items(),
            key=lambda x: x[1]['PV_Utility'],
            reverse=True
        )
        
        for scenario, metrics in sorted_scenarios:
            analysis += f"""
        - {scenario}:
            * PV Utility: {metrics['PV_Utility']:.2f}
            * Steady State Output: {metrics['SS_Output']:.2f}
            * Steady State Consumption: {metrics['SS_Consumption']:.2f}
            * Half-life to Steady State: {metrics['Half_Life']} periods
            """
        
        # Add policy recommendations
        analysis += """
        #### 2. Policy Implications:
        """
        
        # Compare with base scenario
        base_welfare = welfare_analysis['base']['PV_Utility']
        for scenario, metrics in welfare_analysis.items():
            if scenario != 'base':
                welfare_diff = ((metrics['PV_Utility'] - base_welfare) / 
                              abs(base_welfare) * 100)
                
                if welfare_diff > 1:
                    analysis += f"""
        - {scenario}: Welfare gain of {welfare_diff:.1f}% relative to base case
          * Consider policies to move toward this scenario
                    """
                elif welfare_diff < -1:
                    analysis += f"""
        - {scenario}: Welfare loss of {abs(welfare_diff):.1f}% relative to base case
          * Avoid policies leading to this scenario
                    """
        
        display(Markdown(analysis))

# Create interactive policy analysis
analyzer = GrowthPolicyAnalyzer()

# Create widget to select scenarios
@interact
def analyze_policy_scenarios(
    scenarios=widgets.SelectMultiple(
        options=list(analyzer.scenarios.keys()),
        value=['base', 'high_saving', 'high_tech'],
        description='Scenarios:',
        style={'description_width': 'initial'},
        layout=Layout(width='500px')
    )
):
    analyzer.compare_scenarios(list(scenarios))

# --- Model Summary ---

# 🧠 Solow Model Equation

The **Solow Growth Model** in per effective worker terms is:

$$
\dot{k} = s k^\alpha - (\delta + g_A + g_L) k
$$

Where:
- $k$ = capital per effective worker  
- $s$ = savings rate  
- $\delta$ = depreciation rate  
- $g_A$ = growth rate of technology  
- $g_L$ = population growth rate  
- $\alpha$ = output elasticity of capital

The **steady state** level of $k^*$ satisfies:

$$
k^* = \left(\frac{s}{\delta + g_A + g_L}\right)^{\frac{1}{1 - \alpha}}
$$
